In [2]:
import pandas as pd
import networkx as nx
import markov_clustering as mc
import numpy as np
import os

# Task 2.1

# a)

Calculate the following global (i.e. concerning the whole network and not the single nodes)
measures of SGI, U and I (only if no. of nodes >20):

• No. of nodes and no. of links

• No. of connected components

• No. of isolated nodes

• Average path length

• Average degree

• Average clustering coefficient

• Network diameter & radius

• Centralization

In [3]:
# read union dataset
union_df = pd.read_csv('results/union.csv', index_col=0)


In [4]:
intersection_df = pd.read_csv('results/intersection.csv', index_col=0)

In [5]:
sgi_df = pd.read_csv('results/sgi.csv', index_col=0)

In [5]:
def task_2_1_a(df, type_net):
    # undirected graph object
    graph = nx.from_pandas_edgelist(df, source = 'interactor A gene symbol', target='interactor B gene symbol')
    # check number of nodes
    if graph.number_of_nodes() >20:
        n_nodes = graph.number_of_nodes()
        # not consider the duplicates (union can have same edges with different sources: they will be considered just one time)
        n_edges = graph.number_of_edges()
        # number of connected components
        conn_components = nx.number_connected_components(graph)
        # number of isolates
        n_isolates = nx.number_of_isolates(graph)
        # average degree
        avg_degree = n_edges/n_nodes
        # avg clustering coefficient
        avg_cluster_coeff = nx.average_clustering(graph)
        
        print('%s'%type_net,'network has: \n')
        print(n_nodes, 'nodes')
        print(n_edges, 'edges')
        print(conn_components,'connected components')
        print(n_isolates, 'isolated nodes')
        print('average degree = ', avg_degree)
        print('average clustering coefficient = ', avg_cluster_coeff)
        
        # if graph is connected
        if conn_components ==1:
            
            # average shprtest path length
            avg_path = nx.average_shortest_path_length(graph)
            # diameter 
            diameter = nx.diameter(graph)
            # radius 
            radius = nx.radius(graph)
        
            print('shortest path length = ', avg_path)
            print('diameter = ', diameter)
            print('radius = ', radius)
            
        # if graph not connected  
        else:
            ll=[]
            #for each connected component computes the properties
            c = 1
            for g in nx.connected_component_subgraphs(graph): 
                print('Connected Component',c)
                print('average Shortest Path: ', nx.average_shortest_path_length(g))
                print('diameter', nx.diameter(g))
                print('radius', nx.radius(g))
                c +=1
    else:
        print('%s'%type_net,'network do not have a number of edges bigger than 20')

In [6]:
task_2_1_a(intersection_df, 'Intersection')

Intersection network has: 

57 nodes
89 edges
10 connected components
0 isolated nodes
average degree =  1.5614035087719298
average clustering coefficient =  0.09359509885825675
Connected Component 1
average Shortest Path:  3.6659619450317127
diameter 9
radius 5
Connected Component 2
average Shortest Path:  0
diameter 0
radius 0
Connected Component 3
average Shortest Path:  0
diameter 0
radius 0
Connected Component 4
average Shortest Path:  1.0
diameter 1
radius 1
Connected Component 5
average Shortest Path:  1.3333333333333333
diameter 2
radius 1
Connected Component 6
average Shortest Path:  0
diameter 0
radius 0
Connected Component 7
average Shortest Path:  1.0
diameter 1
radius 1
Connected Component 8
average Shortest Path:  0
diameter 0
radius 0
Connected Component 9
average Shortest Path:  0
diameter 0
radius 0
Connected Component 10
average Shortest Path:  0
diameter 0
radius 0


In [7]:
task_2_1_a(union_df, 'Union')

Union network has: 

5513 nodes
10329 edges
2 connected components
0 isolated nodes
average degree =  1.8735715581353165
average clustering coefficient =  0.07321978067098309
Connected Component 1
average Shortest Path:  3.4924265343579193
diameter 7
radius 4
Connected Component 2
average Shortest Path:  1.5
diameter 2
radius 1


In [8]:
task_2_1_a(sgi_df, 'SGI')

SGI network has: 

74 nodes
176 edges
13 connected components
0 isolated nodes
average degree =  2.3783783783783785
average clustering coefficient =  0.15667418167418168
Connected Component 1
average Shortest Path:  3.23879781420765
diameter 7
radius 4
Connected Component 2
average Shortest Path:  0
diameter 0
radius 0
Connected Component 3
average Shortest Path:  0
diameter 0
radius 0
Connected Component 4
average Shortest Path:  0
diameter 0
radius 0
Connected Component 5
average Shortest Path:  0
diameter 0
radius 0
Connected Component 6
average Shortest Path:  1.0
diameter 1
radius 1
Connected Component 7
average Shortest Path:  0
diameter 0
radius 0
Connected Component 8
average Shortest Path:  0
diameter 0
radius 0
Connected Component 9
average Shortest Path:  0
diameter 0
radius 0
Connected Component 10
average Shortest Path:  0
diameter 0
radius 0
Connected Component 11
average Shortest Path:  0
diameter 0
radius 0
Connected Component 12
average Shortest Path:  0
diameter 0
rad

# b) 

Isolate the largest connected component (LCC) of I and U and calculate the following global and local (i.e. for each node) measures:

### Global:

• N. of nodes and no. of links

• Average path length

• Average degree

• Average clustering coefficient

• Network diameter & radius

• Centralization

### Local:

• Node degree

• Betweenness centrality

• Eigenvector centrality

• Closeness centrality

• ratio Betweenness/Node degree


Store the results in a suitable matrix format of your choice.

In [64]:
'''
class LCC

Save to csv the adjacency matrix in order to use it on R.

Compute global and local measures of the largest connected component 
    of the graph corresponding to the given df.
'''

class LCC(object):
    def __init__(self, dataframe, data_type):
        '''
        - dataframe: dataset corresponding to the union or the intersection
        
        - data_type: string to indicate type of df ('union' or 'intersection')
        '''
        
        self.df = dataframe
        self.data_type = data_type
        
    
    def lcc_a_matrix_to_csv(self):
        '''
        Returns:
        
            - file .csv with the adjacency matrix of the graph corresponding 
                to the df given in input to the class. 
        
        It is necessary saving the matrix on an external file in order to use it for more 
            computations in R, where specific functions are faster.
        '''
    
        # create graph from df
        graph = nx.from_pandas_edgelist(self.df, source = 'interactor A gene symbol', target='interactor B gene symbol')
        # return set of nodes of the largest connected component
        lcc_set = max(nx.connected_components(graph), key=len)
        # lcc subgraph
        lcc_graph = graph.subgraph(lcc_set)
        # numpy adj matrix
        a= nx.to_numpy_matrix(lcc_graph)
        try:
            os.remove('data/%s'%self.data_type +'_lcc_matrix.csv')
        except:
            pass
        # save to csv
        #np.savetxt('data/%s'%data_type+'_lcc_matrix.csv', a, delimiter=",", header='')
        pd.DataFrame(a).to_csv('data/%s'%self.data_type +'_lcc_matrix.csv')

    def task_2_1_b_global(self):
        '''
        Returns:
        
            - df: dataframe of global measures for the graph which corresponds to the 
                input dataframe (union or inters).
            
            - nodes: list of nodes names of the graph.
        '''
        # create graph from df
        graph = nx.from_pandas_edgelist(self.df, source = 'interactor A gene symbol', target='interactor B gene symbol')
        # return set of nodes of the largest connected component
        lcc_set = max(nx.connected_components(graph), key=len)
        # lcc subgraph
        lcc_graph = graph.subgraph(lcc_set)
        # number of nodes
        n_nodes = lcc_graph.number_of_nodes()
        # number of edges
        # not consider the duplicates (union can have same edges with different sources: they will be considered just one time)
        n_edges = lcc_graph.number_of_edges()
        # average degree
        avg_degree = n_edges/n_nodes
        # avg clustering coefficient
        avg_cluster_coeff = nx.average_clustering(lcc_graph)
        # create df
        df = pd.DataFrame([n_nodes, n_edges, avg_degree, avg_cluster_coeff])
        df.rename({0:'Nodes', 1:'Edges', 2:'Avg Degree', 3:'Avg Clustering Coeff'}, inplace = True)
        # lcc nodes
        nodes = list(lcc_graph.nodes())
        return df, nodes

    def merge_global_measures(self):
        '''
        Return:
        
            - dataframe containing the global measures of the graph computed on Python and R.
                In particular there are: number of edges and nodes, avg degree, avg clustering coefficient, 
                    avg. path length, diameter and radius. 
        
        '''
        # compute the above function to obtain global measures
        lcc_global1, lcc_nodes  = task_2_1_b_global(self.df, self.data_type)
        # upload the df containg avg shortest path, diameter and radius
        # it was computed on R because of the better computational time of the functions
        lcc_global2 = pd.read_csv('data/%s'%self.data_type+'_lcc_global_results.csv')
        # concatenate the 2 dataframe in one
        lcc_global = pd.concat([lcc_global1.T, lcc_global2], axis = 1).drop(['Unnamed: 0'], axis=1)
        lcc_global = lcc_global.rename({0:'lcc_%s'%self.data_type}) 
        
        return lcc_global
    
    def save_local_results(self):
        _,lcc_nodes = self.task_2_1_b_global()
        # load local dfs computed on R because of the better computational time
        lcc_local = pd.read_csv('data/%s'%self.data_type +'_lcc_local_results.csv').drop('Unnamed: 0', axis=1)
        # rename rows with gene names
        lcc_local = lcc_local.rename(dict(zip([x for x in range(len(lcc_local))],lcc_nodes)))
        # save local df results
        try:
            os.remove('results/%s'%self.data_type +'_lcc_local.csv')
        except:
            pass

        lcc_local.to_csv('results/%s'%self.data_type +'_lcc_local.csv')

In [65]:
# call the class for the 2 dataframes
LCC_union = LCC(union_df, 'union')
LCC_inters = LCC(intersection_df, 'intersection')

# save adjacency matrix for the 2 graphs in order to use it on R
LCC_union.lcc_a_matrix_to_csv()
LCC_inters.lcc_a_matrix_to_csv()

### TASK 2.1 B - GLOBAL
# union global measures df 
lcc_union_global = LCC_union.merge_global_measures()
# intersection global measures df 
lcc_inters_global = LCC_inters.merge_global_measures()
# concatenate the 2 final dataframes
lcc_global = pd.concat([lcc_union_global, lcc_inters_global], axis = 0)
# save resulting df on csv
# remove if already existing adn then create the new one
try:
    os.remove('results/lcc_global.csv')
except: 
    pass
lcc_global.to_csv('results/lcc_global.csv')


### TASK 2.1 B - LOCAL

# save lcc union local results
LCC_union.save_local_results()
# save lcc intersection local results
LCC_inters.save_local_results()



# Task 2.2

Cluster I-LCC and U-LCC using the MCL algorithm to get the modules.
Once you have clustered the networks, find modules with no. of nodes >= 10 in which seed genes are statistically overrepresented (p<0.05) by applying a hypergeometric test: such modules will be the “putative disease modules”.
Store the results for both U-LCC and I-LCC in tables including in each row: clustering algorithm used, module ID, no. of seed genes in the module, total no. of genes in each module, seed gene IDs, all gene IDs in the module, p-value.